In [70]:
import pandas as pd
import numpy as np
from datetime import date

In [71]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [72]:
# Loading the data from csv file that we've got in the previous step SQL analysis:
indexes = pd.read_csv('/content/drive/MyDrive/all_perks.csv')
indexes.head()

,user_id,hotel_hunter_index,average_bags_scaled,cancellation_rate_scaled,bargain_hunter_index,combined_booking_scaled,session_intencity_index
0,101961.0,NaN,0.080000,0.0,0.000058,1.0,0.0
1,106907.0,NaN,1.000000,0.5,NaN,1.0,0.0
2,181157.0,0.000495,0.200000,0.0,0.004943,1.0,0.0
3,190866.0,NaN,0.080000,0.0,NaN,1.0,0.0
4,204997.0,0.000898,0.066667,0.4,0.011774,0.8,0.0



### **Approach to Customer Segmentation:**


1.   We will begin by ranking all customers for each of the perks based on their respective indexes in descending order.


2.   After ranking, we will determine the minimum rank for each customer, and label them according to the perk associated with the minimum rank.

3. We will create a function to determine the rank based on the minimum rank encountered

4. In cases where customers have multiple minimum ranks, we will assign them the label corresponding to the first minimum rank encountered.

5. Customers who have shown no activity in booking and possess all NaN values for indexes will be categorized as "Active Searchers." We recommend keeping these customers in this segment for a period to better understand their preferences before making perk suggestions. We will also include in this group customers whose minimum rank is Session Activity, so that we will have customers with non-null indexes. This will help us gain insights into the preferences of this segment.

In [73]:
# Step 1. Ranking all our indexes in descending order, keeping NaNs:
indexes['rank_hotel_hunter'] = indexes['hotel_hunter_index'].rank(na_option = 'keep', ascending=False)
indexes['rank_ave_bags'] = indexes['average_bags_scaled'].rank(na_option = 'keep', ascending=False)
indexes['rank_cancellation_rank'] = indexes['cancellation_rate_scaled'].rank(na_option = 'keep', ascending=False)
indexes['rank_bargain_hunter'] = indexes['bargain_hunter_index'].rank(na_option = 'keep', ascending=False)
indexes['rank_combined_booking'] = indexes['combined_booking_scaled'].rank(na_option = 'keep', ascending=False)
indexes['rank_session_activity'] = indexes['session_intencity_index'].rank(na_option = 'keep', ascending=False)
indexes.head(5)

,user_id,hotel_hunter_index,average_bags_scaled,cancellation_rate_scaled,bargain_hunter_index,combined_booking_scaled,session_intencity_index,rank_hotel_hunter,rank_ave_bags,rank_cancellation_rank,rank_bargain_hunter,rank_combined_booking,rank_session_activity
0,101961.0,NaN,0.080000,0.0,0.000058,1.0,0.0,NaN,3380.0,3069.0,1679.0,1233.0,3000.0
1,106907.0,NaN,1.000000,0.5,NaN,1.0,0.0,NaN,4.0,52.0,NaN,1233.0,3000.0
2,181157.0,0.000495,0.200000,0.0,0.004943,1.0,0.0,827.0,913.5,3069.0,467.0,1233.0,3000.0
3,190866.0,NaN,0.080000,0.0,NaN,1.0,0.0,NaN,3380.0,3069.0,NaN,1233.0,3000.0
4,204997.0,0.000898,0.066667,0.4,0.011774,0.8,0.0,619.0,3647.5,104.5,203.0,2675.5,3000.0


In [74]:
# Step 2. Finding minimum rating for each user
indexes['min_c_h'] = indexes[['rank_hotel_hunter',
                              'rank_ave_bags',
                              'rank_cancellation_rank',
                              'rank_bargain_hunter',
                              'rank_combined_booking',
                              'rank_session_activity']].min(axis=1)
indexes.head()

,user_id,hotel_hunter_index,average_bags_scaled,cancellation_rate_scaled,bargain_hunter_index,combined_booking_scaled,session_intencity_index,rank_hotel_hunter,rank_ave_bags,rank_cancellation_rank,rank_bargain_hunter,rank_combined_booking,rank_session_activity,min_c_h
0,101961.0,NaN,0.080000,0.0,0.000058,1.0,0.0,NaN,3380.0,3069.0,1679.0,1233.0,3000.0,1233.0
1,106907.0,NaN,1.000000,0.5,NaN,1.0,0.0,NaN,4.0,52.0,NaN,1233.0,3000.0,4.0
2,181157.0,0.000495,0.200000,0.0,0.004943,1.0,0.0,827.0,913.5,3069.0,467.0,1233.0,3000.0,467.0
3,190866.0,NaN,0.080000,0.0,NaN,1.0,0.0,NaN,3380.0,3069.0,NaN,1233.0,3000.0,1233.0
4,204997.0,0.000898,0.066667,0.4,0.011774,0.8,0.0,619.0,3647.5,104.5,203.0,2675.5,3000.0,104.5


In [75]:
# Step 3. Creating function for defining the most attractive perk for each user:
def perk_define(row):
    if row['rank_hotel_hunter'] == row['min_c_h']:
        perk = 'Free Hotel Meal'
    elif row['rank_ave_bags'] == row['min_c_h']:
        perk = 'Free Checked Bag'
    elif row['rank_cancellation_rank'] == row['min_c_h']:
        perk = 'No Cancellation Fee'
    elif row['rank_bargain_hunter'] == row['min_c_h']:
        perk = 'Exclusive Discount'
    elif row['rank_combined_booking'] == row['min_c_h']:
        perk = 'Night Free Hotel with Flight'
    elif row['rank_session_activity'] == row['min_c_h']:
        perk = 'Active Searcher'
    else:
        'Not Defined'
    return perk



In [76]:
# Step 4: Applying our function to determine customers' ranks:
indexes['most_attractive_perk'] = indexes.apply(perk_define, axis=1)
indexes.head()

,user_id,hotel_hunter_index,average_bags_scaled,cancellation_rate_scaled,bargain_hunter_index,combined_booking_scaled,session_intencity_index,rank_hotel_hunter,rank_ave_bags,rank_cancellation_rank,rank_bargain_hunter,rank_combined_booking,rank_session_activity,min_c_h,most_attractive_perk
0,101961.0,NaN,0.080000,0.0,0.000058,1.0,0.0,NaN,3380.0,3069.0,1679.0,1233.0,3000.0,1233.0,Night Free Hotel with Flight
1,106907.0,NaN,1.000000,0.5,NaN,1.0,0.0,NaN,4.0,52.0,NaN,1233.0,3000.0,4.0,Free Checked Bag
2,181157.0,0.000495,0.200000,0.0,0.004943,1.0,0.0,827.0,913.5,3069.0,467.0,1233.0,3000.0,467.0,Exclusive Discount
3,190866.0,NaN,0.080000,0.0,NaN,1.0,0.0,NaN,3380.0,3069.0,NaN,1233.0,3000.0,1233.0,Night Free Hotel with Flight
4,204997.0,0.000898,0.066667,0.4,0.011774,0.8,0.0,619.0,3647.5,104.5,203.0,2675.5,3000.0,104.5,No Cancellation Fee


In [77]:
# Let's see how many users in each segment
indexes['most_attractive_perk'].value_counts()

,count
most_attractive_perk,
Free Checked Bag,1354
Free Hotel Meal,1072
Night Free Hotel with Flight,1047
Active Searcher,1034
Exclusive Discount,1000
No Cancellation Fee,491


In [78]:
# Step 5: Downloading the file for further analysis in Tableau:
indexes.to_csv('Data_with_Perks.csv')

# Part 2. Processing users data

In [79]:
# Loading the data from csv file with users data:
users_data = pd.read_csv('/content/drive/MyDrive/user_data.csv')
users_data.head()

,user_id,birthdate,gender,married,has_children,home_country,home_city,home_airport,home_airport_lat,home_airport_lon
0,23557,1958-12-08,F,True,False,usa,new york,LGA,40.777,-73.872
1,94883,1972-03-16,F,True,False,usa,kansas city,MCI,39.297,-94.714
2,101486,1972-12-07,F,True,True,usa,tacoma,TCM,47.138,-122.476
3,101961,1980-09-14,F,True,False,usa,boston,BOS,42.364,-71.005
4,106907,1978-11-17,F,True,True,usa,miami,TNT,25.862,-80.897


In [80]:
users_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5998 entries, 0 to 5997
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   user_id           5998 non-null   int64  
 1   birthdate         5998 non-null   object 
 2   gender            5998 non-null   object 
 3   married           5998 non-null   bool   
 4   has_children      5998 non-null   bool   
 5   home_country      5998 non-null   object 
 6   home_city         5998 non-null   object 
 7   home_airport      5998 non-null   object 
 8   home_airport_lat  5998 non-null   float64
 9   home_airport_lon  5998 non-null   float64
dtypes: bool(2), float64(2), int64(1), object(5)
memory usage: 386.7+ KB


We have identified a few issues in our dataset that require attention:

1. Convert 'Birthdate' to date format.
2. Calculate the age based on 'Birthdate'.
3. Check the 'Gender' column for the number of unique values and consider converting this data to boolean.


In [92]:
# Convert Birthdate to date format:
users_data['birthdate'] = pd.to_datetime(users_data['birthdate'], errors='coerce')

# Calculating age:
users_data['age'] = (pd.Timestamp.now() - users_data['birthdate']).dt.days // 365.25
users_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5998 entries, 0 to 5997
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   user_id           5998 non-null   int64         
 1   birthdate         5998 non-null   datetime64[ns]
 2   gender            5998 non-null   object        
 3   married           5998 non-null   bool          
 4   has_children      5998 non-null   bool          
 5   home_country      5998 non-null   object        
 6   home_city         5998 non-null   object        
 7   home_airport      5998 non-null   object        
 8   home_airport_lat  5998 non-null   float64       
 9   home_airport_lon  5998 non-null   float64       
 10  age               5998 non-null   float64       
dtypes: bool(2), datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 433.6+ KB


In [93]:
# Cheking values in column Gender
users_data['gender'].value_counts()

,count
gender,
F,5292
M,695
O,11


In [94]:
# Since amount of O-values is insignificant (only 0.18% of total) we will replace them by most frequent
# value ("F") and convert this column to boolean

replace_values = {'F' : True, 'O' : True, 'M' : False }
users_data['gender'] = users_data['gender'].map(replace_values)

In [98]:
print(users_data.columns)

Index(['user_id', 'gender', 'married', 'has_children', 'age'], dtype='object')


In [109]:
# We will drop unnecessary columns from the 'users_data' DataFrame to keep only the relevant information.
users_data.drop(['birthdate', 'home_country',
                 'home_city', 'home_airport',
                 'home_airport_lat', 'home_airport_lon',
                 'sign_up_date'], axis='columns', inplace=True, errors='ignore')


In [110]:
users_data.head()

,married,has_children
0,True,False
1,True,False
2,True,True
3,True,False
4,True,True


In [112]:
# Downloading file for further analysis in Tableau:
users_data.to_csv('/content/drive/MyDrive/Colab Notebooks/users_data_processed.csv')